Auswertung der Parquet Dateien

In [1]:
import duckdb
import pandas as pd
import para
import openpyxl
from lonboard import Map, HeatmapLayer
import datetime as dt

In [2]:
pd.options.display.max_columns = 100

In [3]:
jetzt = dt.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
gestern= (dt.date.today() - dt.timedelta(1)).strftime('%Y-%m-%d')
letzte07tage= (dt.date.today() - dt.timedelta(7)).strftime('%Y-%m-%d')
letzte14tage= (dt.date.today() - dt.timedelta(14)).strftime('%Y-%m-%d')
letzte21tage= (dt.date.today() - dt.timedelta(21)).strftime('%Y-%m-%d')

print(jetzt, letzte21tage)

2024-09-16 08:09:14 2024-08-26


In [4]:
con = duckdb.connect()

## Verbindung zur DM Datenbank und Import der Grunddaten

In [5]:
con.sql(f"""INSTALL postgres;
LOAD postgres;
ATTACH 'dbname=zvbn_postgis user=postgres host=127.0.0.1 password={para.key_dm_db}' AS db_dm (TYPE POSTGRES, READ_ONLY);""")

In [6]:
con.sql("create or replace table lin_buendel as select * from db_dm.basis.lin_buendel")
con.sql("select * from lin_buendel")

┌─────────────────────┬────────────┬───────────────────────┬───────┬───────────────────────────────────────────┬────────────┬────────┬─────────┬─────────────────────┬────────────────────────────────────────────────────────────────┬─────────────┬───────────────────┐
│       buendel       │   harmon   │         kreis         │  id   │                 bemerkung                 │ kreis_kurz │ lfd_nr │  aktiv  │    buendel_lang     │                          project_vms                           │ rt_operator │  betreiber_kurz   │
│       varchar       │    date    │        varchar        │ int32 │                  varchar                  │  varchar   │ int32  │ boolean │       varchar       │                            varchar                             │   varchar   │      varchar      │
├─────────────────────┼────────────┼───────────────────────┼───────┼───────────────────────────────────────────┼────────────┼────────┼─────────┼─────────────────────┼────────────────────────────────────

In [7]:
sql_lin = """
        Create or replace table linien as 
        SELECT nummer AS linie, buendel, ebene, dlid, id 
        FROM db_dm.basis.linien 
        WHERE buendel IS NOT NULL AND aktiv IS TRUE 
        ORDER BY buendel, ebene, nummer """
con.sql(sql_lin)
con.sql("select * from linien")

┌─────────┬─────────┬─────────┬───────────────┬───────┐
│  linie  │ buendel │  ebene  │     dlid      │  id   │
│ varchar │ varchar │ varchar │    varchar    │ int32 │
├─────────┼─────────┼─────────┼───────────────┼───────┤
│ 330     │ AM Ost  │ 1+      │ de:VBN:330    │     7 │
│ 340     │ AM Ost  │ 1+      │ de:VBN:340    │   348 │
│ 370     │ AM Ost  │ 2       │ de:VBN:370    │    59 │
│ 331     │ AM Ost  │ 3       │ de:VBN:331    │   287 │
│ 332     │ AM Ost  │ 3       │ de:VBN:332    │   231 │
│ 333     │ AM Ost  │ 3       │ de:VBN:333    │   232 │
│ 334     │ AM Ost  │ 3       │ de:VBN:334    │   233 │
│ 335     │ AM Ost  │ 3       │ de:VBN:335    │   234 │
│ 336     │ AM Ost  │ 3       │ de:VBN:336    │   235 │
│ 337     │ AM Ost  │ 3       │ de:VBN:337    │   360 │
│  ·      │   ·     │ ·       │     ·         │    ·  │
│  ·      │   ·     │ ·       │     ·         │    ·  │
│  ·      │   ·     │ ·       │     ·         │    ·  │
│ 448     │ WM Süd  │ 3       │ de:VBN:448    │ 

In [8]:
server = 'prod'
con.sql(f"create or replace table fahrten as select * from read_parquet('out/parquet/{server}/fahrten*.parquet',  union_by_name = true, filename = true)")
con.sql(f"create or replace table verlauf as select * from read_parquet('out/parquet/{server}/verlauf*.parquet',  union_by_name = true, filename = true)")
con.sql(f"create or replace table zusatz as select * from read_parquet('out/parquet/{server}/zusatz*.parquet',  union_by_name = true, filename = true)")

In [9]:
con.sql(f"select count(*) from fahrten where datum >= '{letzte14tage}'")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       210874 │
└──────────────┘

In [10]:
print(con.sql("select count(*) from fahrten"),con.sql("select count(*) from verlauf")) 

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│       263227 │
└──────────────┘
 ┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│      6266125 │
└──────────────┘



In [11]:
con.sql("""select 
            datum, 
            fahrtstartstationname, 
           strftime( cast(fahrtstarttime as TIMESTAMPTZ), '%H:%M') as fahrtstart,
           fahrtendstationname,
           strftime( cast(fahrtendtime as TIMESTAMPTZ), '%H:%M') as fahrtende,
            
            deviceid, 
            split_part(deviceid, '-', 2) as fnr, 
            cast(((cast(split_part(split_part(deviceid, '-', 3), '#', 1) as int64) - 8000000000000) / 1000) as int64) as m2, 
        from fahrten 
        where deviceid like '%680%DBRB%' and datum = '2024-08-29'
        order by datum, fahrtstarttime
        
        """).df()
#.to_excel('out/web.xlsx', index=False)

,datum,fahrtstartstationname,fahrtstart,fahrtendstationname,fahrtende,deviceid,fnr,m2
0,2024-08-29,Wallhöfen Am Mühlenberg,04:08,Bremen Gröpelingen,05:36,0829-1680002-8012306400000#!ADD!#DBRB#,1680002,12306400
1,2024-08-29,Bremen Gröpelingen,05:08,Wallhöfen Am Mühlenberg,06:32,0829-1680001-8012303800000#!ADD!#DBRB#,1680001,12303800
2,2024-08-29,Wallhöfen Am Mühlenberg,05:24,Bremen Gröpelingen,06:52,0829-1680004-8012306401000#!ADD!#DBRB#,1680004,12306401
3,2024-08-29,Vollersode-Giehlermühlen,06:20,Wallhöfen Am Mühlenberg,06:46,0829-1686602-8012316800000#!ADD!#DBRB#,1686602,12316800
4,2024-08-29,Wallhöfen Am Mühlenberg,06:46,Bremen Gröpelingen,08:26,0829-1680006-8012308200000#!ADD!#DBRB#,1680006,12308200
5,2024-08-29,Bremen-Burg(Bus),06:53,Osterholz-Scharmbeck Bahnhof (Bus),07:40,0829-1680005-8012305800000#!ADD!#DBRB#,1680005,12305800
6,2024-08-29,Wallhöfen Am Mühlenberg,07:46,Bremen Gröpelingen,09:21,0829-1680008-8012308300000#!ADD!#DBRB#,1680008,12308300
7,2024-08-29,Osterholz-Scharmbeck Bahnhof (Bus),08:00,Bremen-Burg(Bus),08:39,0829-1680302-8012307400000#!ADD!#DBRB#,1680302,12307400
8,2024-08-29,Bremen Gröpelingen,08:08,Wallhöfen Am Mühlenberg,09:32,0829-1680007-8012303700000#!ADD!#DBRB#,1680007,12303700
9,2024-08-29,Wallhöfen Am Mühlenberg,08:24,Bremen Gröpelingen,09:52,0829-1680010-8012306500000#!ADD!#DBRB#,1680010,12306500


In [12]:
con.sql("select  journeyOperator, count(journeyOperator) as count from verlauf group by journeyOperator order by count")

┌─────────────────────────────────────────────────────────────┬─────────┐
│                       journeyOperator                       │  count  │
│                           varchar                           │  int64  │
├─────────────────────────────────────────────────────────────┼─────────┤
│ Autobus Stoss GmbH                                          │    5606 │
│ SBV Janßen GmbH & Co. KG                                    │    7870 │
│ Regionalverkehre Start Deutschland (Niedersachsen-Mitte)    │   10607 │
│ WestfalenBahn                                               │   11200 │
│ Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH(Bahndaten) │   15162 │
│ eurobahn                                                    │   15379 │
│ Weser-Ems-Bus Auftragnehmerleistungen                       │   18835 │
│ metronom Eisenbahngesellschaft mbH                          │   19409 │
│ W. Giese Nachf. Omnibusbetrieb GmbH                         │   33447 │
│ Delmenhorst-Harpstedter Eisenbahn Gm

## Fahrten mit hohen Verspätungen

In [13]:
con.sql("select distinct deviceid from verlauf where dep_del > 100").df()

,deviceid
0,0903-20387#!ADD!#eurobahn#
1,0903-82275/20240903#!ADD!#NWB-LS##!ADD!#tdg-pr...
2,0907-6032003#!ADD!#IVU-Regio#
3,0910-1633003#!ADD!#IVU-Regio#
4,0910-1280073#!ADD!#IVU-Regio#
...,...
131,0903-1283007#!ADD!#IVU-Regio#
132,0903-1283010#!ADD!#IVU-Regio#
133,0904-3739027-00027-1#!ADD!#BSAG#
134,0912-82016#!ADD!#me#


In [14]:
con.sql("describe fahrten")

┌─────────────────────────────┬──────────────┬─────────┬─────────┬─────────┬─────────┐
│         column_name         │ column_type  │  null   │   key   │ default │  extra  │
│           varchar           │   varchar    │ varchar │ varchar │ varchar │ varchar │
├─────────────────────────────┼──────────────┼─────────┼─────────┼─────────┼─────────┤
│ datum                       │ TIMESTAMP_NS │ YES     │ NULL    │ NULL    │ NULL    │
│ fnr                         │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ destination                 │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ hasRealtime                 │ BOOLEAN      │ YES     │ NULL    │ NULL    │ NULL    │
│ vu                          │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ lineid                      │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ lineid_short                │ VARCHAR      │ YES     │ NULL    │ NULL    │ NULL    │
│ lineshort                   │ VARCHAR    

### Verkürzung der DLID
- Zum Teil weren bei mehreren Betreibern einer Linie TLID mit vierteiliger DLID geliefert 
- Verkürzung ermöglicht die Verknüpfung mit Liste aus DM

In [15]:
con.sql("alter table fahrten add column if not exists lineid_short VARCHAR")
con.sql("""update fahrten 
        set lineid_short = concat_ws(':', split_part(lineid,':', 1), split_part(lineid,':', 2), split_part(lineid,':', 3))""")
con.sql("""select distinct lineid, 
        concat_ws(':', split_part(lineid,':', 1), split_part(lineid,':', 2), split_part(lineid,':', 3)) 
        from fahrten""")

┌───────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────┐
│    lineid     │ concat_ws(':', split_part(lineid, ':', 1), split_part(lineid, ':', 2), split_part(lineid, ':', 3)) │
│    varchar    │                                              varchar                                               │
├───────────────┼────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ de:VBN:95:    │ de:VBN:95                                                                                          │
│ de:VBN:20:1   │ de:VBN:20                                                                                          │
│ de:VBN:S:VGB  │ de:VBN:S                                                                                           │
│ de:VBN:4S:    │ de:VBN:4S                                                                                          │
│ de:VBN:AST41: │ de:VBN:AST41                  

## Über HIM gemeldete Ausfälle

In [16]:
df_fahrten_ausfall_him = con.sql(f"""
                              select vu, fnr, ts_reported_cancelled, journey_cancelled 
                              from fahrten f 
                              where ts_reported_cancelled != '' and f.datum >= '{letzte14tage}'""").df()

## Echzeitquote

### nach Linie und Betreiber

In [17]:
df_ez_quote_betreiber = con.sql(f"""
        select l.buendel, l.ebene,f.datum, f.vu, f.lineshort,f.lineid_short, count(f.hasRealtime) filter (f.hasRealtime = True) ez_true, count(f.*) count, 
        round(ez_true / count * 100, 1) anteil_ez
        from fahrten f
        left outer join linien l on f.lineid_short = l.dlid
        where f.datum >= '{letzte14tage}'              
        group by f.lineid_short, f.vu, f.datum, f.lineshort, f.lineid_short, l.buendel, l.ebene
        order by f.vu, f.lineid_short
        """).df()
df_ez_quote_betreiber['buendel'] = df_ez_quote_betreiber['buendel'].fillna('-')
df_ez_quote_betreiber['ebene'] = df_ez_quote_betreiber['ebene'].fillna('-')
anteil_ez_pivot_betreiber = pd.pivot_table(df_ez_quote_betreiber, index=['buendel','ebene', 'vu', 'lineshort'], columns='datum', values='anteil_ez').reset_index()
anteil_ez_pivot_betreiber

datum,buendel,ebene,vu,lineshort,2024-09-02 00:00:00,2024-09-03 00:00:00,2024-09-04 00:00:00,2024-09-05 00:00:00,2024-09-06 00:00:00,2024-09-07 00:00:00,2024-09-08 00:00:00,2024-09-09 00:00:00,2024-09-10 00:00:00,2024-09-11 00:00:00,2024-09-12 00:00:00,2024-09-13 00:00:00,2024-09-14 00:00:00,2024-09-15 00:00:00
0,-,-,Autobus Stoss GmbH,558,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,-,-,Autobus Stoss GmbH,559,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,-,-,BREMERHAVEN BUS,518,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-,-,BREMERHAVEN BUS,519,0.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0,0.0,0.0,NaN,NaN
4,-,-,BREMERHAVEN BUS,E01,100.0,100.0,100.0,100.0,100.0,NaN,NaN,100.0,100.0,100.0,100.0,100.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
676,WM Süd,3,Gebken Reisen GmbH,458,92.0,100.0,100.0,96.0,90.9,NaN,NaN,96.0,100.0,96.0,100.0,90.9,NaN,NaN
677,WM Süd,3,Gebken Reisen GmbH,459,76.5,70.6,41.2,76.5,64.3,NaN,NaN,70.6,76.5,76.5,76.5,50.0,NaN,NaN
678,WM Süd,3,Gebken Reisen GmbH,461,80.0,90.0,90.0,90.0,88.9,NaN,NaN,90.0,90.0,80.0,80.0,88.9,NaN,NaN
679,WM Süd,3,Gebken Reisen GmbH,462,66.7,100.0,100.0,100.0,100.0,NaN,NaN,100.0,100.0,100.0,100.0,100.0,NaN,NaN


### nach Linie (ohne Betreiber)

In [18]:
df_ez_quote_o_betreiber = con.sql(f"""
        select l.buendel, l.ebene,f.datum, f.lineshort,f.lineid_short, count(f.hasRealtime) filter (f.hasRealtime = True) ez_true, count(f.*) count, 
        round(ez_true / count * 100, 1) anteil_ez
        from fahrten f        
        left outer join linien l on f.lineid_short = l.dlid      
        where f.datum >= '{letzte14tage}'        
        group by f.lineid_short, f.datum, f.lineshort, f.lineid_short, l.buendel, l.ebene
        order by f.lineid_short
        """).df()
df_ez_quote_o_betreiber['buendel'] = df_ez_quote_o_betreiber['buendel'].fillna('-')
df_ez_quote_o_betreiber['ebene'] = df_ez_quote_o_betreiber['ebene'].fillna('-')
anteil_ez_pivot_o_betreiber = pd.pivot_table(df_ez_quote_o_betreiber, index=['buendel','ebene', 'lineshort'], columns='datum', values='anteil_ez').reset_index()
anteil_ez_pivot_o_betreiber

datum,buendel,ebene,lineshort,2024-09-02 00:00:00,2024-09-03 00:00:00,2024-09-04 00:00:00,2024-09-05 00:00:00,2024-09-06 00:00:00,2024-09-07 00:00:00,2024-09-08 00:00:00,2024-09-09 00:00:00,2024-09-10 00:00:00,2024-09-11 00:00:00,2024-09-12 00:00:00,2024-09-13 00:00:00,2024-09-14 00:00:00,2024-09-15 00:00:00
0,-,-,1020,100.0,100.0,100.0,100.0,100.0,100.0,NaN,100.0,100.0,100.0,100.0,100.0,0.0,NaN
1,-,-,1021,96.2,96.2,96.2,96.2,65.4,66.7,100.0,96.2,92.3,88.5,88.5,96.2,0.0,0.0
2,-,-,108,100.0,100.0,100.0,97.0,100.0,NaN,NaN,100.0,100.0,100.0,100.0,100.0,NaN,NaN
3,-,-,10E,100.0,100.0,100.0,100.0,97.4,99.5,100.0,100.0,100.0,66.7,100.0,100.0,NaN,NaN
4,-,-,1E,98.1,100.0,100.0,100.0,100.0,100.0,100.0,98.1,98.1,92.5,98.1,100.0,100.0,100.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
639,WM Süd,3,458,92.0,100.0,100.0,96.0,90.9,NaN,NaN,96.0,100.0,96.0,100.0,90.9,NaN,NaN
640,WM Süd,3,459,76.5,70.6,41.2,76.5,64.3,NaN,NaN,70.6,76.5,76.5,76.5,50.0,NaN,NaN
641,WM Süd,3,461,80.0,90.0,90.0,90.0,88.9,NaN,NaN,90.0,90.0,80.0,80.0,88.9,NaN,NaN
642,WM Süd,3,462,66.7,100.0,100.0,100.0,100.0,NaN,NaN,100.0,100.0,100.0,100.0,100.0,NaN,NaN


## Klären der Problemfälle

### Fahren der 95 die nicht als hasRealtime gekennzeichnet sind

In [19]:
con.sql("select * from fahrten where lineshort = 'S35' and datum = '2024-09-05'").df().query("hasRealtime == False")

,datum,fnr,destination,hasRealtime,vu,lineid,lineid_short,lineshort,reported_cancelled,journey_cancelled,ts_reported_cancelled,cancelled_kum,deviceid,clientid,journeyrttype,fahrtstarttime,fahrtstartstationname,fahrtstartstationdhid,fahrtendtime,fahrtendstationname,fahrtendstationdhid,realtimeHasEverBeenReported,filename
15,2024-09-05,6352001,Oldenburg ZOB,False,Gerdes Reisen,de:VBN:S35:,de:VBN:S35,S35,False,False,,False,,-,SCHEDULED,2024-09-05T09:35:00+02:00,Westerstede Hössenbad,de:03451:98641::1,2024-09-05T10:20:00+02:00,Oldenburg(Oldb) ZOB,de:03403:27022:3:5,NaN,out/parquet/prod/fahrten_2024_09_05.parquet
17,2024-09-05,6352924,Westerstede,False,Gerdes Reisen,de:VBN:S35:,de:VBN:S35,S35,False,False,,False,,-,SCHEDULED,2024-09-05T10:35:00+02:00,Oldenburg(Oldb) ZOB,de:03403:27022:3:8,2024-09-05T11:12:00+02:00,Westerstede Ammerland-Klinik,de:03451:98623::1,NaN,out/parquet/prod/fahrten_2024_09_05.parquet
20,2024-09-05,6352005,Oldenburg ZOB,False,Gerdes Reisen,de:VBN:S35:,de:VBN:S35,S35,False,False,,False,,-,SCHEDULED,2024-09-05T11:35:00+02:00,Westerstede Hössenbad,de:03451:98641::1,2024-09-05T12:20:00+02:00,Oldenburg(Oldb) ZOB,de:03403:27022:3:5,NaN,out/parquet/prod/fahrten_2024_09_05.parquet
22,2024-09-05,6352030,Westerstede,False,Gerdes Reisen,de:VBN:S35:,de:VBN:S35,S35,False,False,,False,,-,SCHEDULED,2024-09-05T12:35:00+02:00,Oldenburg(Oldb) ZOB,de:03403:27022:3:8,2024-09-05T13:12:00+02:00,Westerstede Ammerland-Klinik,de:03451:98623::1,NaN,out/parquet/prod/fahrten_2024_09_05.parquet


In [20]:
con.sql("select * from verlauf where operday = '2024-08-29' and ex_lineid = 'de:VBN:95:' and fnr = '6534'").df().head()

,operday,journeyOperator,deviceid,lineshortname,ex_lineid,fnr,index,has_rt,dschedtime,aschedtime,dep_del,arr_del,station_nr,station_name,lat,lon,canc,additional,ts_reported_cancelled,reported_cancelled,filename
0,2024-08-29,Bremer Straßenbahn AG,0829-5280018-00018-1#!ADD!#BSAG#,95,de:VBN:95:,6534,1,False,2024-08-29 14:35:00,NaT,NaN,NaN,623142,Bremen Gröpelingen,53.120972,8.752869,False,False,,False,out/parquet/prod/verlauf_2024_08_29.parquet
1,2024-08-29,Bremer Straßenbahn AG,0829-5280018-00018-1#!ADD!#BSAG#,95,de:VBN:95:,6534,2,False,2024-08-29 14:37:00,2024-08-29 14:37:00,NaN,NaN,606322,Bremen Karl-Bröger-Straße,53.125134,8.748617,False,False,,False,out/parquet/prod/verlauf_2024_08_29.parquet
2,2024-08-29,Bremer Straßenbahn AG,0829-5280018-00018-1#!ADD!#BSAG#,95,de:VBN:95:,6534,3,False,2024-08-29 14:37:00,2024-08-29 14:37:00,NaN,NaN,108492,Bremen DIAKO Bremen,53.126671,8.746846,False,False,,False,out/parquet/prod/verlauf_2024_08_29.parquet
3,2024-08-29,Bremer Straßenbahn AG,0829-5280018-00018-1#!ADD!#BSAG#,95,de:VBN:95:,6534,4,False,2024-08-29 14:38:00,2024-08-29 14:38:00,NaN,NaN,606021,Bremen Am Fuchsberg,53.129646,8.743403,False,False,,False,out/parquet/prod/verlauf_2024_08_29.parquet
4,2024-08-29,Bremer Straßenbahn AG,0829-5280018-00018-1#!ADD!#BSAG#,95,de:VBN:95:,6534,5,False,2024-08-29 14:40:00,2024-08-29 14:40:00,NaN,NaN,108682,Bremen Ritterhuder Heerstraße,53.134015,8.735564,False,False,,False,out/parquet/prod/verlauf_2024_08_29.parquet


In [21]:
con.sql("select * from (select vu, datum, lineshort, count(lineshort) count from zusatz group by vu, datum, lineshort) where count > 10 order by count desc").df().to_excel('out/zusatz.xlsx', index=False)

## Fahrten ohne Echtzeit Ebene 1/1+ und 2

In [22]:
df_fahrten_ohne_ez_ebenen_1_1p_2 = con.sql(f"""
        select f.datum, l.buendel, l.ebene, f.vu, f.fnr, f.lineshort,f.lineid_short, f.hasrealtime, f.journey_cancelled, f.reported_cancelled, f.ts_reported_cancelled
        
        from fahrten f
        left outer join linien l on f.lineid_short = l.dlid              
        where l.ebene in ('1', '1+') and f.hasrealtime = False and f.datum >= '{letzte14tage}'
                                           
        order by f.datum, f.lineid_short
        """).df()

In [23]:
df_fahrten_ausfall_1_1p_2 = con.sql(f"""
        select f.datum, l.buendel, l.ebene, f.vu, f.fnr, f.lineshort,f.lineid_short, f.hasrealtime, f.journey_cancelled, f.reported_cancelled, f.ts_reported_cancelled
        
        from fahrten f
        left outer join linien l on f.lineid_short = l.dlid              
        where l.ebene in ('1', '1+', '2') and (journey_cancelled = True or f.reported_cancelled = True) and 
        f.datum >= '{letzte14tage}'                            
        order by f.datum, f.lineid_short
        """).df()
df_fahrten_ausfall_1_1p_2.head()

,datum,buendel,ebene,vu,fnr,lineshort,lineid_short,hasRealtime,journey_cancelled,reported_cancelled,ts_reported_cancelled
0,2024-09-03,OHZ Ost,1+,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,1630037,630,de:VBN:630,False,False,True,2024-09-03T14:00:04.990+02:00
1,2024-09-03,OHZ Ost,1+,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,1630040,630,de:VBN:630,False,False,True,2024-09-03T16:29:03.161+02:00
2,2024-09-04,OHZ Mitte,2,Weser-Ems-Bus Betrieb Bremen,1660003,660,de:VBN:660,False,True,True,2024-09-04T10:06:59.147+02:00
3,2024-09-04,VER Südwest,1,Weser-Ems-Bus Auftragnehmerleistungen,1740061,740,de:VBN:740,True,True,True,2024-09-04T20:10:58.748+02:00
4,2024-09-07,OHZ Ost,1+,Eisenbahnen und Verkehrsbetriebe Elbe-Weser GmbH,1630127,630,de:VBN:630,False,False,True,2024-09-07T18:52:01.395+02:00


In [24]:
#df_fahrten_ohne_ez_ebenen_1_1p_2.tail(5)
df_fahrten_ohne_ez_ebenen_1_1p_2[['vu', 'fnr']].value_counts().reset_index().sort_values(['count', 'vu'], ascending=False)

,vu,fnr,count
0,Reisedienst von Rahden GmbH & Co. KG,1740039,11
6,Verkehrsbetriebe Wesermarsch GmbH,6400008,10
7,Reisedienst von Rahden GmbH & Co. KG,1740027,10
8,Reisedienst von Rahden GmbH & Co. KG,1740063,10
12,Reisedienst von Rahden GmbH & Co. KG,1740059,10
...,...,...,...
178,Bruns Reisen GmbH Bad Zwischenahn,380048,1
217,Bruns Reisen GmbH Bad Zwischenahn,380017,1
218,Bruns Reisen GmbH Bad Zwischenahn,380020,1
219,Bruns Reisen GmbH Bad Zwischenahn,380025,1


# Ausgabe Anteil EZ nach Linie

In [25]:
xlsx = "/var/www/rt_archiv/anteil_echtzeit_linien_vbn.xlsx"
sn00 = '00 Hilfe'
sn01 = '01 pivot alle Linien betreiber'
sn02 = '02 pivot alle Linien'
sn03 = '03 fahrten ohne EZ 1 1+ 2'
sn04 = '04 fahrten ohne EZ 1 1+ 3 grup'
sn06 = '05 fahrten ausfall'
sn07 = '06 fahrten ausfall über HIM'
with pd.ExcelWriter(xlsx, engine="openpyxl") as writer:
    #Hilfeblatt
    writer.book.create_sheet(sn00)
    sheet = writer.book[sn00]
    sheet['A1'] = f"Erstellt: {dt.datetime.now().strftime('%Y-%m-%d %H:%M')} Zeitraum: {letzte14tage} bis {gestern}"

    sheet['A3'] = "Inhalt"
    sheet['B4'] = f"Blatt {sn01}: Pivot Echtzeitquote inkl. Betreiberkennung"
    sheet['B5'] = f"Blatt {sn02}: Pivot Echtzeitquote ohne Betreiberkennung"
    sheet['B6'] = f"Blatt {sn03}: Fahrten ohne Echtzeit"
    sheet['B7'] = f"Blatt {sn04}: Fahrten ohne Echtzeit mit Anzahl"
    sheet['B8'] = f"Blatt {sn06}: Fahrten Ausfall"
    sheet['B9'] = f"Blatt {sn07}: Fahrten Ausfall über HIM"

    #mit Kennung der Betreiber
    anteil_ez_pivot_betreiber.to_excel(writer, sheet_name=sn01, index=False)
    writer.book[sn01].freeze_panes = 'e2'
    writer.book[sn01].auto_filter.ref='A:H'
    for cell in writer.book[sn01]["1:1"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn01].column_dimensions['c'].width = 22
    for c in ['D', 'E', 'F', 'G', 'H']:
        writer.book[sn01].column_dimensions[c].width = 22        
    for c in writer.book[sn01].iter_cols(min_col=4, max_col=anteil_ez_pivot_betreiber.shape[1]+4):
                #ermitteln der Spalte column letter
                cl = c[int(f"{anteil_ez_pivot_betreiber.shape[0]}")].column_letter
                writer.book[sn01].column_dimensions[cl].width = 16

    #Anteil EZ ohne Kennung der Betreiber
    anteil_ez_pivot_o_betreiber.to_excel(writer, sheet_name=sn02, index=False)
    writer.book[sn02].freeze_panes = 'd2'
    writer.book[sn02].auto_filter.ref='A:H'
    for cell in writer.book[sn02]["1:1"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn02].column_dimensions['c'].width = 22
    for c in ['D', 'E', 'F', 'G', 'H']:
        writer.book[sn02].column_dimensions[c].width = 22 
         
    for c in writer.book[sn02].iter_cols(min_col=4, max_col=anteil_ez_pivot_o_betreiber.shape[1]+4):
                #ermitteln der Spalte column letter
                cl = c[int(f"{anteil_ez_pivot_o_betreiber.shape[0]}")].column_letter
                writer.book[sn02].column_dimensions[cl].width = 16

    ## Ausgabe der Fahrten ohne Echtzeit Ebene 1 und 1+ und 2 einzeln
    df_fahrten_ohne_ez_ebenen_1_1p_2.to_excel(writer, sheet_name=sn03, index=False)
    writer.book[sn03].freeze_panes = 'a2'
    writer.book[sn03].auto_filter.ref='A:M'
    for cell in writer.book[sn03]["A"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn03].column_dimensions['A'].width = 18

    ## Ausgabe der Fahrten ohne Echtzeit Ebene 1 und 1+ und 2 gruppiert mit Anzahl
    df_fahrten_ohne_ez_ebenen_1_1p_2[['vu', 'fnr']].value_counts().reset_index().sort_values(['count', 'vu'], ascending=False).to_excel(writer, sheet_name=sn04, index=False)
    writer.book[sn04].freeze_panes = 'a2'
    writer.book[sn04].auto_filter.ref='A:H'
    writer.book[sn04].column_dimensions['A'].width = 22   

    ## Ausgabe der Fahrten Ausfall Ebene 1, 1+ und 2
    df_fahrten_ausfall_1_1p_2.to_excel(writer, sheet_name=sn06, index=False)
    writer.book[sn06].freeze_panes = 'a2'
    writer.book[sn06].auto_filter.ref='A:M'
    for cell in writer.book[sn06]["A"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn06].column_dimensions['A'].width = 18

    ## Ausgabe der Fahrten Ausfall über HIM
    df_fahrten_ausfall_him.to_excel(writer, sheet_name=sn07, index=False)
    writer.book[sn07].freeze_panes = 'a2'
    writer.book[sn07].auto_filter.ref='A:M'
    for cell in writer.book[sn07]["A"]:
        cell.number_format = 'YYYY-MM-DD'
    writer.book[sn07].column_dimensions['A'].width = 18